In [ ]:
import tkinter as tk
import sqlite3

# -------- Database Setup --------
conn = sqlite3.connect("todo.db")
cursor = conn.cursor()
cursor.execute("""
CREATE TABLE IF NOT EXISTS tasks (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    task TEXT NOT NULL
)
""")
conn.commit()

# -------- Functions --------
def load_tasks():
    listbox.delete(0, tk.END)  # Clear listbox
    cursor.execute("SELECT id, task FROM tasks")
    for row in cursor.fetchall():
        listbox.insert(tk.END, f"{row[0]} - {row[1]}")

def add_task():
    task = entry.get()
    if task:
        cursor.execute("INSERT INTO tasks (task) VALUES (?)", (task,))
        conn.commit()
        entry.delete(0, tk.END)
        load_tasks()

def update_task():
    try:
        selected = listbox.get(listbox.curselection())
        task_id = selected.split(" - ")[0]
        new_task = entry.get()
        if new_task:
            cursor.execute("UPDATE tasks SET task=? WHERE id=?", (new_task, task_id))
            conn.commit()
            entry.delete(0, tk.END)
            load_tasks()
    except:
        pass

def delete_task():
    try:
        selected = listbox.get(listbox.curselection())
        task_id = selected.split(" - ")[0]
        cursor.execute("DELETE FROM tasks WHERE id=?", (task_id,))
        conn.commit()
        load_tasks()
    except:
        pass

# -------- UI Setup --------
root = tk.Tk()
root.title("Simple To-Do List")

entry = tk.Entry(root, width=30)
entry.pack(pady=5)

btn_frame = tk.Frame(root)
btn_frame.pack()

tk.Button(btn_frame, text="Add", width=10, command=add_task).grid(row=0, column=0, padx=2)
tk.Button(btn_frame, text="Update", width=10, command=update_task).grid(row=0, column=1, padx=2)
tk.Button(btn_frame, text="Delete", width=10, command=delete_task).grid(row=0, column=2, padx=2)

listbox = tk.Listbox(root, width=40)
listbox.pack(pady=5)

# Load tasks at start
load_tasks()

root.mainloop()
